In [1]:
import math

import pandas as pd
import numpy as np
import time
import copy
import itertools
import random
import matplotlib.pyplot as plt

In [2]:

class Item:

    def __init__(self):  # 初始化函数，定义几个参数
        self.start = []
        self.end = []
        self._on = []
        self.T = []
        self.last_ot = 0  # 最后一个工序的完成时间
        self.L = 0

    def update(self, s, e, on, t):  # 类中的函数，更新状态
        self.start.append(s)  # 记录工序的开始时间列表
        self.end.append(e)  # 记录工序的结束时间列表
        self._on.append(on)  # 记录加工工序的机器编号（无论是jobs还是machine_list
        self.T.append(t)  # 加工的时间序列
        self.last_ot = e  # 最后的时间为end
        self.L += t  # 时间累加，即机器的负载时间

    @property
    def on(self):
        return self._on


class Scheduling:
    def __init__(self, job_processnum, machine, pt):
        self.M = machine  # 机器集合，[0, 1, 2, 3, 4, 5]
        self.Job_num = len(job_processnum)
        self.Job_processnum = job_processnum  # 各个工件的工序数量列表
        self.pt = pt  # 加工时间
        self.Jobs = []
        self.Machine_list = []  # Jobs和Machine_list 需要在createJob()上方先定义
        self.createJob()
        self.createMachine()
        self.makespan = 0
        self.energycost = 0
        self.maxload = 0
        self.empty_power = [0.6, 0.6, 0.5, 0.4, 0.4, 0.6]  # 空载功率
        self.load_power = [2, 1.6, 1.8, 2.4, 2.4, 4.1]  # 负载功率

    def createJob(self):
        for i in range(self.Job_num):
            J = Item()
            self.Jobs.append(J)

    def createMachine(self):
        for i in range(len(self.M)):
            M_i = Item()
            self.Machine_list.append(M_i)

    def decode(self, Chromo_Job, Chromo_machine):  # 解码主要对Jobs和Machine_list进行更新
        jobcount = [-1 for i in range(self.Job_num)]
        begintime = 0
        endtime = 0
        loadtime = 0
        for i in range(len(Chromo_Job)):
            jobcount[Chromo_Job[i]] += 1
            process_time = \
                self.pt[Chromo_Job[i]][jobcount[Chromo_Job[i]]][Chromo_machine[i]]
            if (not self.Jobs[Chromo_Job[i]].start) and (not self.Machine_list[Chromo_machine[i]].start):
                endtime = begintime + process_time
            elif (not self.Jobs[Chromo_Job[i]].start) and self.Machine_list[Chromo_machine[i]].start:
                begintime = self.Machine_list[Chromo_machine[i]].end[-1]
                endtime = begintime + process_time
            elif self.Jobs[Chromo_Job[i]].start and (not self.Machine_list[Chromo_machine[i]].start):
                begintime = self.Jobs[Chromo_Job[i]].end[-1]
                endtime = begintime + process_time
            else:
                begintime = max(self.Jobs[Chromo_Job[i]].end[-1], self.Machine_list[Chromo_machine[i]].end[-1])
                endtime = begintime + process_time
            loadtime = process_time
            self.Jobs[Chromo_Job[i]].update(begintime, endtime, Chromo_machine[i], loadtime)
            self.Machine_list[Chromo_machine[i]].update(begintime, endtime, Chromo_machine[i], loadtime)

        # 计算makespan、负载、能耗

        self.makespan = max([self.Jobs[i].end[-1] for i in range(self.Job_num)])
        self.maxload = max([self.Machine_list[i].L for i in range(len(self.M))])
        for i in range(len(self.M)):
            if not self.Machine_list[i].end:  # 当该机器未排产时，赋值0
                eneryconsumption = 0
            else:
                eneryconsumption = self.Machine_list[i].L * self.load_power[i] + \
                               (self.Machine_list[i].end[-1] - self.Machine_list[i].start[0] - self.Machine_list[i].L) \
                               * self.empty_power[i]
            self.energycost += eneryconsumption
        return [self.makespan, self.maxload, self.energycost]

    def gant(self):
        return 0

In [4]:
# 初始化，输入参数
path = r'D:\cangmu\Documents\CODE\learn_python\NSGA-II\NSGA-II_FJSP' + '\\FJSP-data\\' + 'MK01' + '.xlsx'
para_tmp = pd.read_excel(path, index_col=[0])
Job_processnum = list(map(int, para_tmp.iloc[:, 0]))
Job_index = list(map(int, para_tmp.index))
Job_num = len(Job_index)
Machine = list(range(6))
popsize = 50  # 设置种群数量
# 数据写入pt
pt = []
for j in range(Job_num):
    p = Job_processnum[j]
    ptarr = [[math.inf for i in range(len(Machine))] for i in range(p)]
    ptlist = list(map(int, para_tmp.iloc[j][1:].dropna()))
    process = 0
    while ptlist != []:
        num = ptlist[0]  # 2
        for i in range(num):
            ptarr[process][ptlist[2 * i + 1] - 1] = ptlist[2 * i + 2]
        del ptlist[0:num * 2 + 1]
        process += 1
    pt.append(ptarr)

pt_index = [[[] for j in range(Job_processnum[i])] for i in range(Job_num)]
pt_worktime = [[[] for j in range(Job_processnum[i])] for i in range(Job_num)]
for i in range(Job_num):
    for j in range(Job_processnum[i]):
        for k in range(len(Machine)):
            if pt[i][j][k] != math.inf:
                pt_index[i][j].append(k)
                pt_worktime[i][j].append(pt[i][j][k])

In [5]:
def index_of(a, list):
    for i in range(0, len(list)):
        if list[i] == a:
            return i
    return -1


def gerenate_chromo(Job_processnum, pi, pt, pt_index):
    Chromo_job = []
    m = 0
    for i in Job_processnum:
        c1 = [m for j in range(i)]
        Chromo_job.extend(c1)
        m += 1
    np.random.shuffle(Chromo_job)
    judge_index = [np.random.random() for i in range(len(Chromo_job))]  # 55个随机数（0,1）
    Chromo_machine = [0 for i in range(len(Chromo_job))]  # 55，用于记录Chromo对应的加工机器的编号
    index_1 = [-1 for i in range(len(Job_processnum))]  # [-1, -1, -1,......]
    for i in range(len(judge_index)):
        index_1[Chromo_job[i]] += 1
        if judge_index[i] <= pi:
            Chromo_machine[i] = index_of(min(pt[Chromo_job[i]][index_1[Chromo_job[i]]][:]),
                                         pt[Chromo_job[i]][index_1[Chromo_job[i]]][:])
        else:
            Chromo_machine[i] = random.choice(pt_index[Chromo_job[i]][index_1[Chromo_job[i]]][:])
    return [Chromo_job, Chromo_machine]



In [9]:
def fintness_calculation(poplation_list):
    chromo_obj_record = []
    for chromo in poplation_list:
        schedule = Scheduling(job_processnum=Job_processnum, machine=Machine, pt=pt)
        chromo_obj_record.append(schedule.decode(Chromo_Job=chromo[0],
                                                 Chromo_machine=chromo[1]))
    return chromo_obj_record


#
def dominate(p, q, chroms_obj_record):
    if (chroms_obj_record[p][0] < chroms_obj_record[q][0] and chroms_obj_record[p][1] < chroms_obj_record[q][1]
        and chroms_obj_record[p][2] < chroms_obj_record[q][2]) or (
            chroms_obj_record[p][0] <= chroms_obj_record[q][0] and chroms_obj_record[p][1] <
            chroms_obj_record[q][1]
            and chroms_obj_record[p][2] < chroms_obj_record[q][2]) or (
            chroms_obj_record[p][0] <= chroms_obj_record[q][0] and chroms_obj_record[p][1] <=
            chroms_obj_record[q][1]
            and chroms_obj_record[p][2] < chroms_obj_record[q][2]) or (
            chroms_obj_record[p][0] <= chroms_obj_record[q][0] and chroms_obj_record[p][1] <
            chroms_obj_record[q][1]
            and chroms_obj_record[p][2] <= chroms_obj_record[q][2]) or (
            chroms_obj_record[p][0] < chroms_obj_record[q][0] and chroms_obj_record[p][1] <=
            chroms_obj_record[q][1]
            and chroms_obj_record[p][2] < chroms_obj_record[q][2]) or (
            chroms_obj_record[p][0] < chroms_obj_record[q][0] and chroms_obj_record[p][1] <=
            chroms_obj_record[q][1]
            and chroms_obj_record[p][2] <= chroms_obj_record[q][2]) or (
            chroms_obj_record[p][0] < chroms_obj_record[q][0] and chroms_obj_record[p][1] <
            chroms_obj_record[q][1]
            and chroms_obj_record[p][2] <= chroms_obj_record[q][2]):
        return 1
    else:
        return 0

# 进行快速非支配排序
def fast_non_dominate_sorting(popsize, chromos_obj_record):
    s, n = {}, {}
    front, rank = {}, {}
    front[0] = []
    iter_range = popsize  # 如果不是亲子代混合，就是100+x个需要比较
    for p in range(iter_range):
        s[p] = []
        n[p] = 0
        for q in range(iter_range):
            if dominate(p, q, chromos_obj_record):  # if p dominates q for three objectives
                if q not in s[p]:
                    s[p].append(q)  # s[p] is the set of solutions dominated by p
            elif dominate(q, p, chromos_obj_record):  # if q dominates p for three objectives
                n[p] = n[p] + 1  # n[p] is the set of solutions dominating p, 3 obj
        if n[p] == 0:
            rank[p] = 0  # p belongs to front 0
            if p not in front[0]:
                front[0].append(p)

    i = 0
    while front[i]:
        Q = []  # Used to store the members of the next front
        for p in front[i]:
            for q in s[p]:
                n[q] = n[q] - 1
                if n[q] == 0:
                    rank[q] = i + 1
                    if q not in Q:
                        Q.append(q)
        i = i + 1
        front[i] = Q

    del front[len(front) - 1]
    return front, rank



In [10]:
# 主函数
Population_list = []
Chromos_obj_record = []
for i in range(popsize):
    Population_list.append(gerenate_chromo(Job_processnum, 0.5, pt, pt_index))
# 解码计算每个个体的适应度，添加至列表
Chromos_obj_record = fintness_calculation(Population_list)
front, rank = fast_non_dominate_sorting(popsize=popsize, chromos_obj_record=Chromos_obj_record)
print(front)
print(rank)

{0: [6, 33, 35, 40], 1: [13, 4, 23, 24, 31, 44, 48, 29], 2: [8, 36, 42, 49, 9, 25, 34, 14, 16, 26], 3: [11, 27, 38, 18, 7, 15, 21, 47, 43], 4: [30, 22, 12, 10, 19, 45, 1], 5: [17, 37, 0, 3, 28, 32], 6: [41, 20, 5, 46], 7: [39], 8: [2]}
{6: 0, 33: 0, 35: 0, 40: 0, 13: 1, 4: 1, 23: 1, 24: 1, 31: 1, 44: 1, 48: 1, 29: 1, 8: 2, 36: 2, 42: 2, 49: 2, 9: 2, 25: 2, 34: 2, 14: 2, 16: 2, 26: 2, 11: 3, 27: 3, 38: 3, 18: 3, 7: 3, 15: 3, 21: 3, 47: 3, 43: 3, 30: 4, 22: 4, 12: 4, 10: 4, 19: 4, 45: 4, 1: 4, 17: 5, 37: 5, 0: 5, 3: 5, 28: 5, 32: 5, 41: 6, 20: 6, 5: 6, 46: 6, 39: 7, 2: 8}


In [17]:
popfun = np.array([Chromos_obj_record[i] for i in range(popsize)])
K = 5
popfun.shape[0]
popfun

array([[ 88. ,  86. , 506.4],
       [ 89. ,  76. , 506. ],
       [119. , 106. , 523.4],
       [ 92. ,  85. , 511.9],
       [ 69. ,  69. , 440.8],
       [ 92. ,  85. , 543.8],
       [ 84. ,  58. , 465.3],
       [ 82. ,  75. , 495.7],
       [ 84. ,  69. , 454.9],
       [ 78. ,  70. , 504.5],
       [ 82. ,  76. , 542.3],
       [ 85. ,  74. , 468.8],
       [ 95. ,  80. , 467.2],
       [ 90. ,  61. , 508.4],
       [ 80. ,  64. , 516.9],
       [ 87. ,  73. , 498.1],
       [ 89. ,  82. , 444.6],
       [ 92. ,  87. , 471.8],
       [ 83. ,  80. , 470.9],
       [ 84. ,  82. , 500.3],
       [ 95. ,  93. , 509.3],
       [ 84. ,  69. , 522.1],
       [ 90. ,  76. , 467.3],
       [ 69. ,  68. , 473.1],
       [ 78. ,  67. , 464.5],
       [ 77. ,  73. , 532.8],
       [ 84. ,  82. , 451.1],
       [ 90. ,  74. , 461. ],
       [ 92. ,  80. , 520.5],
       [ 84. ,  76. , 421.6],
       [ 89. ,  79. , 482.6],
       [ 81. ,  75. , 436.6],
       [ 94. ,  78. , 520. ],
       [ 6

In [21]:
popfun1 = popfun[:(popsize - K), :]
popfun2 = popfun[(popsize - K):, :]
print(popfun1)
print(popfun2)

[[ 88.   86.  506.4]
 [ 89.   76.  506. ]
 [119.  106.  523.4]
 [ 92.   85.  511.9]
 [ 69.   69.  440.8]
 [ 92.   85.  543.8]
 [ 84.   58.  465.3]
 [ 82.   75.  495.7]
 [ 84.   69.  454.9]
 [ 78.   70.  504.5]
 [ 82.   76.  542.3]
 [ 85.   74.  468.8]
 [ 95.   80.  467.2]
 [ 90.   61.  508.4]
 [ 80.   64.  516.9]
 [ 87.   73.  498.1]
 [ 89.   82.  444.6]
 [ 92.   87.  471.8]
 [ 83.   80.  470.9]
 [ 84.   82.  500.3]
 [ 95.   93.  509.3]
 [ 84.   69.  522.1]
 [ 90.   76.  467.3]
 [ 69.   68.  473.1]
 [ 78.   67.  464.5]
 [ 77.   73.  532.8]
 [ 84.   82.  451.1]
 [ 90.   74.  461. ]
 [ 92.   80.  520.5]
 [ 84.   76.  421.6]
 [ 89.   79.  482.6]
 [ 81.   75.  436.6]
 [ 94.   78.  520. ]
 [ 64.   62.  450.4]
 [ 79.   69.  483.2]
 [ 69.   63.  404.6]
 [ 88.   74.  451.1]
 [ 93.   82.  469. ]
 [ 94.   80.  458.5]
 [106.  100.  490.6]
 [ 76.   76.  402.5]
 [ 98.   92.  487.4]
 [ 79.   75.  465.2]
 [ 92.   87.  455.9]
 [ 77.   68.  473. ]]
[[ 83.   80.  518.8]
 [ 99.   86.  529.1]
 [ 81.   68.

In [27]:
a = np.array([[5,1,2],[1,2,9],[1,9,7],[5,4,1],[5,7,9]])
b = np.max(a,0)
print(b)
b.reshape(3,1)
b.T

[5 9 9]


array([5, 9, 9])